### Create yeobuckner7
Atlas combines 7-network Yeo Cerebral atlas with the 1-network Buckner Cerebellar atlas. Areas of overlap between cerebellum and cerebrum are resolved by assigning cerebellar values to the overlapping voxels.

In [1]:
import nibabel as nib
import numpy as np
import pandas as pd

In [2]:
def reorient(input_file, ori, out_file=None):
    from nipype.interfaces.image import Reorient
    from nipype import Node
    import os

    reorient_node = Node(Reorient(),name="reorient_to_{}".format(ori))
    reorient_node.inputs.in_file =  os.path.abspath(input_file)
    reorient_node.inputs.orientation=ori
    reorient_results = reorient_node.run()
    results_file = os.path.abspath(reorient_results.outputs.out_file)
    if out_file:
        out_file = os.path.abspath(out_file)
        os.system(f"mv {results_file} {out_file}")
    else:
        out_file = results_file
    
    return out_file

In [3]:
# https://github.com/ThomasYeoLab/CBIG/blob/master/stable_projects/brain_parcellation/Yeo2011_fcMRI_clustering/1000subjects_reference/Yeo_JNeurophysiol11_SplitLabels/Yeo2011_7networks_N1000.split_components.glossary.csv
label_file = "./sources/Originals/Yeo2011_7networks_N1000.split_components.glossary.csv"
yeo_df = pd.read_table(label_file,sep=",")

In [4]:
yeo_df

,Label Name,Network Name,Full component name
0,NONE,NaN,NaN
1,7Networks_LH_Vis,visual,visual
2,7Networks_LH_SomMot,somatomotor,somatomotor
3,7Networks_LH_DorsAttn_Post,dorsal attention,posterior
4,7Networks_LH_DorsAttn_FEF,dorsal attention,frontal eye fields
5,7Networks_LH_DorsAttn_PrCv,dorsal attention,precentral ventral
6,7Networks_LH_SalVentAttn_ParOper,salience / ventral attention,parietal operculum
7,7Networks_LH_SalVentAttn_TempOcc,salience / ventral attention,temporal occipital
8,7Networks_LH_SalVentAttn_FrOperIns,salience / ventral attention,frontal operculum insula
9,7Networks_LH_SalVentAttn_PFCl,salience / ventral attention,lateral prefrontal cortex


In [5]:
# ftp://surfer.nmr.mgh.harvard.edu/pub/data/Yeo_JNeurophysiol11_MNI152.zip
# Yeo2011_7Networks_MNI152_FreeSurferConformed1mm_LiberalMask.nii.gz transformed to FSL LAS Space
atlas_file = "./sources/yeo7_1mm.nii.gz"
atlas_img = nib.load(atlas_file)
atlas_data = atlas_img.get_fdata()

In [6]:
# https://github.com/ThomasYeoLab/CBIG/tree/master/stable_projects/brain_parcellation/Yeo2011_fcMRI_clustering/1000subjects_reference/Yeo_JNeurophysiol11_SplitLabels/MNI152
# $PWD/Yeo2011_7Networks_N1000.split_components.FSL_MNI152_FreeSurferConformed_1mm.nii.gz transformed to FSL LAS space
split_file = "./sources/yeo7_splitlabel_1mm.nii.gz"
split_img = nib.load(split_file)
split_data = split_img.get_fdata()

In [7]:
from scipy import stats

for roi in range(1,int(np.max(atlas_data))+1):
    mask=atlas_data == roi
    split_rois=split_data[mask]
    split_rois=split_rois[split_rois != 0]
    modalscore = stats.mode(split_rois)
    labs = yeo_df.iloc[int(modalscore.mode),yeo_df.columns.get_loc("Network Name")]
    print(f"network {roi} has {modalscore.mode}which is {labs}")

network 1 has 27.0which is visual
network 2 has 2.0which is somatomotor
network 3 has 29.0which is dorsal attention
network 4 has 34.0which is salience / ventral attention
network 5 has 39.0which is limbic
network 6 has 43.0which is control
network 7 has 24.0which is default


In [8]:
label_dict = {"Vis": 1,
            "SomMot":2,            
            "DorsAttn":3,     
            "SalVentAttn":4,     
            "Limbic":5,     
            "Cont":6,  
            "Default":7  
}

Above confirmed also at https://github.com/ThomasYeoLab/CBIG/blob/master/stable_projects/brain_parcellation/Yeo2011_fcMRI_clustering/1000subjects_reference/7NetworksOrderedNames.csv

In [9]:
yeo_df

,Label Name,Network Name,Full component name
0,NONE,NaN,NaN
1,7Networks_LH_Vis,visual,visual
2,7Networks_LH_SomMot,somatomotor,somatomotor
3,7Networks_LH_DorsAttn_Post,dorsal attention,posterior
4,7Networks_LH_DorsAttn_FEF,dorsal attention,frontal eye fields
5,7Networks_LH_DorsAttn_PrCv,dorsal attention,precentral ventral
6,7Networks_LH_SalVentAttn_ParOper,salience / ventral attention,parietal operculum
7,7Networks_LH_SalVentAttn_TempOcc,salience / ventral attention,temporal occipital
8,7Networks_LH_SalVentAttn_FrOperIns,salience / ventral attention,frontal operculum insula
9,7Networks_LH_SalVentAttn_PFCl,salience / ventral attention,lateral prefrontal cortex


In [15]:
yeo7_df=yeo_df.copy()
yeo7_df["net7_num"]=0
yeo7_df["net7_name"]=yeo7_df["Network Name"]

In [16]:
yeo7_df

,Label Name,Network Name,Full component name,net7_num,net7_name
0,NONE,NaN,NaN,0,NaN
1,7Networks_LH_Vis,visual,visual,0,visual
2,7Networks_LH_SomMot,somatomotor,somatomotor,0,somatomotor
3,7Networks_LH_DorsAttn_Post,dorsal attention,posterior,0,dorsal attention
4,7Networks_LH_DorsAttn_FEF,dorsal attention,frontal eye fields,0,dorsal attention
5,7Networks_LH_DorsAttn_PrCv,dorsal attention,precentral ventral,0,dorsal attention
6,7Networks_LH_SalVentAttn_ParOper,salience / ventral attention,parietal operculum,0,salience / ventral attention
7,7Networks_LH_SalVentAttn_TempOcc,salience / ventral attention,temporal occipital,0,salience / ventral attention
8,7Networks_LH_SalVentAttn_FrOperIns,salience / ventral attention,frontal operculum insula,0,salience / ventral attention
9,7Networks_LH_SalVentAttn_PFCl,salience / ventral attention,lateral prefrontal cortex,0,salience / ventral attention


In [17]:
replacement_dict = {"visual": "Vis",
            "somatomotor": "SomMot",           
            "dorsal attention": "DorsAttn",     
            "ventral attention": "SalVentAttn",     
            "limbic": "Limbic",     
            "control": "Cont",     
            "default": "Default",  
}

In [18]:
import re
def replace_location(value):
    if not pd.isna(value):
        value = re.sub(r'\s+', ' ', value).strip()
        for key, replacement in replacement_dict.items():
            if key.lower() in value.lower():
                return replacement
    return value

In [19]:
yeo7_df["net7_name"] = yeo7_df["net7_name"].apply(replace_location)

In [21]:
yeo7_df["net7_name"].tolist()

[nan,
 'Vis',
 'SomMot',
 'DorsAttn',
 'DorsAttn',
 'DorsAttn',
 'SalventAttn',
 'SalventAttn',
 'SalventAttn',
 'SalventAttn',
 'SalventAttn',
 'Limbic',
 'Limbic',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Default',
 'Default',
 'Default',
 'Default',
 'Default',
 'Vis',
 'SomMot',
 'DorsAttn',
 'DorsAttn',
 'DorsAttn',
 'SalventAttn',
 'SalventAttn',
 'SalventAttn',
 'SalventAttn',
 'SalventAttn',
 'SalventAttn',
 'Limbic',
 'Limbic',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Cont',
 'Default',
 'Default',
 'Default',
 'Default',
 'Default']

In [23]:
yeo7_df["net7_num"] = yeo7_df["net7_name"].map(label_dict)

In [25]:
yeo7_df

,Label Name,Network Name,Full component name,net7_num,net7_name
0,NONE,NaN,NaN,NaN,NaN
1,7Networks_LH_Vis,visual,visual,1.0,Vis
2,7Networks_LH_SomMot,somatomotor,somatomotor,2.0,SomMot
3,7Networks_LH_DorsAttn_Post,dorsal attention,posterior,3.0,DorsAttn
4,7Networks_LH_DorsAttn_FEF,dorsal attention,frontal eye fields,3.0,DorsAttn
5,7Networks_LH_DorsAttn_PrCv,dorsal attention,precentral ventral,3.0,DorsAttn
6,7Networks_LH_SalVentAttn_ParOper,salience / ventral attention,parietal operculum,4.0,SalventAttn
7,7Networks_LH_SalVentAttn_TempOcc,salience / ventral attention,temporal occipital,4.0,SalventAttn
8,7Networks_LH_SalVentAttn_FrOperIns,salience / ventral attention,frontal operculum insula,4.0,SalventAttn
9,7Networks_LH_SalVentAttn_PFCl,salience / ventral attention,lateral prefrontal cortex,4.0,SalventAttn


### Add Cerebellar atlas

In [26]:
# ftp://surfer.nmr.mgh.harvard.edu/pub/data/Buckner_JNeurophysiol11_MNI152.zip
# Buckner2011_17Networks_MNI152_FreeSurferConformed1mm_LooseMask.nii.gz transformed to FSL LAS Space
buck_file = "./sources/buck7_1mm.nii.gz"
buck_img = nib.load(buck_file)
buck_data = buck_img.get_fdata()

### Where do the Yeo and buckner overlap

In [27]:
mask1=atlas_data > 0
mask2=buck_data > 0
combined_mask = np.logical_and(mask1,mask2)

In [28]:
atlas_data[combined_mask]

array([3., 1., 3., ..., 3., 3., 3.])

In [29]:
stats.mode(buck_data[combined_mask])

ModeResult(mode=4.0, count=1781)

In [30]:
buck_data[combined_mask]

array([6., 6., 6., ..., 6., 6., 6.])

In [31]:
yeo_data = atlas_data + buck_data

In [32]:
yeo_data[combined_mask]=buck_data[combined_mask]
yeo_data[combined_mask]

array([6., 6., 6., ..., 6., 6., 6.])

In [33]:
np.max(yeo_data)

7.0

In [34]:
final_img = nib.Nifti1Image(yeo_data, atlas_img.affine, atlas_img.header)
nib.save(final_img, "./outputs/atlas-yeobuckner7_space-MNI152NLin6Asym_res-01_ori-LAS_dseg.nii.gz")

In [35]:
net7_labeldf = pd.DataFrame(list(label_dict.keys()),columns=["label"])
net7_labeldf["index"] = net7_labeldf["label"].map(label_dict)
net7_labeldf = net7_labeldf[["index","label"]]
net7_labeldf

,index,label
0,1,Vis
1,2,SomMot
2,3,DorsAttn
3,4,SalventAttn
4,5,Limbic
5,6,Cont
6,7,Default


In [36]:
net7_labeldf.to_csv("./outputs/atlas-yeobuckner7_dseg.tsv",sep="\t",header=True, index=False)

In [37]:
input_file = "./outputs/atlas-yeobuckner7_space-MNI152NLin6Asym_res-01_ori-LAS_dseg.nii.gz"
out_file="./outputs/atlas-yeobuckner7_space-MNI152NLin6Asym_res-01_dseg.nii.gz"
final = reorient(input_file, "RAS", out_file)

250215-15:35:37,420 nipype.workflow INFO:
	 [Node] Setting-up "reorient_to_RAS" in "/tmp/tmp2gd21w9m/reorient_to_RAS".
250215-15:35:37,422 nipype.workflow INFO:
	 [Node] Executing "reorient_to_RAS" <nipype.interfaces.image.Reorient>
250215-15:35:37,626 nipype.workflow INFO:
	 [Node] Finished "reorient_to_RAS", elapsed time 0.200254s.


In [38]:
final

'/groups/ryant/PANapps/PANpipelines/PAN2025_Deployment/atlas/yeobuckner/Development/outputs/atlas-yeobuckner7_space-MNI152NLin6Asym_res-01_dseg.nii.gz'